In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import math

## Read the file

There are 3 options to download the data from the WRI ClimateWatch tool. Here we download the data by clicking on "Download Bulk Data", and selecting "GHG Emissions". The dataset is available [here](https://github.com/OpenGeoScales/ogs-data-exploration/tree/main/data/ghg-emissions/wri) and is named :
- CW_UNFCCC_GHG_Emissions.xlsx

In [2]:
# Read datafile corresponding to territorial emissions
df = pd.read_excel("../../../data/ghg-emissions/wri/CW_UNFCCC_GHG_Emissions.xlsx")

# Clean the "sector" column, as for one sector we have different terms depending on UNFCCC_AI or UNFCCC_NAI
df.replace('Land Use, Land-Use Change and Forestry', 'Land-Use Change and Forestry', inplace = True)
df.replace('Total GHG emissions without LULUCF', 'Total GHG emissions excluding LULUCF/LUCF', inplace = True)
df.replace('Total GHG emissions with LULUCF', 'Total GHG emissions including LULUCF/LUCF', inplace = True)
df.replace('Industrial Processes', 'Industrial Processes and Product Use', inplace = True)
df.replace('Solvent and Other Product Use', 'Industrial Processes and Product Use', inplace = True)

# Set the year columns in the same one 
df = df.melt(id_vars=["country", "source", "sector", "gas"], 
        var_name="year", 
        value_name="MtCO2e")

# Set the dataframe structure
df.sort_values(["year","country","sector","gas"],inplace=True)
df.set_index(["year","country","sector","gas"],inplace=True)

df.head()

source  MtCO2e
year country sector      gas                               
1990 AFG     Agriculture Aggregate GHGs  UNFCCC_NAI     NaN
                         CH4             UNFCCC_NAI     NaN
                         CO2             UNFCCC_NAI     NaN
                         N2O             UNFCCC_NAI     NaN
             Energy      Aggregate GHGs  UNFCCC_NAI     NaN

Now that we have the final dataframe structure, we can see that regarding the country AFG, we do not have the gas "Aggregate F-gases". If we do not have that information we could have errors by working on the mapping step, as the loop will go through all the unique sectors and gases that exist in the dataframe. 

## Mapping

In [3]:
# Option to select only a subset of countries/year (to check structure)
#list_years=[2017,2018]
#list_country=["ANNEXI","AFG"]
#list_sectors=["Agriculture", "Energy"]
#list_gas=["CH4", "Aggregate F-gases"]

# Define the range of years, countries, sectors and gases from the data that we will map (here it is the entire dataset). 
list_years = df.index.get_level_values(0).unique()
list_country = df.index.get_level_values(1).unique()
list_sectors = df.index.get_level_values(2).unique()
list_gas = df.index.get_level_values(3).unique()

In [4]:
# Mapping sector
mapped_sectors={}
mapped_sectors["Agriculture"] = "agriculture"
mapped_sectors["Energy"] = "total_energy"
mapped_sectors["Industrial Processes and Product Use"] = "industrial_processes"
mapped_sectors["Land-Use Change and Forestry"] = "lucf"
mapped_sectors["Total GHG emissions excluding LULUCF/LUCF"] = "total_excluding_LUCF"
mapped_sectors["Total GHG emissions including LULUCF/LUCF"] = "total_including_LUCF"
mapped_sectors["Waste"] = "waste"
mapped_sectors["Other"] = "other"

# Mapping gas name
mapped_gas_name={}
mapped_gas_name["Aggregate GHGs"] = "kyotogases"
mapped_gas_name["CH4"] = "CH4"
mapped_gas_name["CO2"] = "CO2"
mapped_gas_name["N2O"] = "N2O"
mapped_gas_name["Aggregate F-gases"] = "F-gas"

In [5]:
# Define the name for the output mapped datafile.
path="../../../data/ghg-emissions/wri/mapped/"
data_name="wri-unfccc"
namefile = path+"mapped_data_"+data_name+".json"

In [6]:
# Create the output file
file = open(namefile, "w")


# Loop over geo_component (in this case, countries)
for country in list_country:
    # Loop over years
    for yr in list_years:
        # Loop over sectors
        for sect in list_sectors:
            # Loop over gases
            for gas in list_gas:

                try:

                    # Get the emission value from dataset
                    emission_value = df.loc[(yr, country, sect, gas), ("MtCO2e")].values[0]

                    if not math.isnan(emission_value):

                        # Create dictionary that will contain the final json object
                        dict_mapped_entry={}

                        ############## Entries associated with data_source
                        dict_data_source={}
                        
                        # Sub-dictionary data_source properties
                        dict_data_source_properties={}
                        dict_data_source_properties["description"]=None
                        dict_data_source_properties["scenario"]=df.loc[(yr, country, sect, gas), ("source")].values[0]
                        
                        dict_data_source["name"]=data_name    
                        dict_data_source["link"]="https://www.climatewatchdata.org/"
                        dict_data_source["properties"]=dict_data_source_properties

                        ############## Entries associated with geo_component
                        dict_geo_component={}

                        # Sub-dictionary geo-component ID
                        dict_geo_component_id={}
                        dict_geo_component_id["id"]=country
                        dict_geo_component_id["type"]="alpha3"

                        dict_geo_component["scale"]="country"    
                        dict_geo_component["identifier"]=dict_geo_component_id

                        ############# Entries associated with emission
                        dict_emission={}

                        # Sub-dictionnary on unit
                        dict_emission_unit={}
                        dict_emission_unit["unit_used"]="Mt co2eq"

                        # Sub-dictionary on sector
                        dict_emission_sector={}
                        dict_emission_sector["sector_origin_name"]=sect
                        dict_emission_sector["sector_mapped_name"]=mapped_sectors[sect]

                        dict_emission["gas"]=mapped_gas_name[gas]
                        dict_emission["value"]=emission_value
                        dict_emission["unit"]=dict_emission_unit
                        dict_emission["sector"]=dict_emission_sector         

                        ############# Combine everything together
                        dict_mapped_entry["data_source"]=dict_data_source
                        dict_mapped_entry["geo_component"]=dict_geo_component 
                        dict_mapped_entry["date"]=str(yr)+"-01-01"
                        dict_mapped_entry["emission"]=dict_emission

                        # Write the json object to a file and add a line break (every line is a json object)
                        json.dump(dict_mapped_entry, file)
                        file.write("\n")

                except:
                    pass

file.close()

In [7]:
df_json = pd.read_json("../../../data/ghg-emissions/wri/mapped/mapped_data_wri-unfccc.json", lines=True)
df_json

,data_source,geo_component,date,emission
0,"{'name': 'wri-unfccc', 'link': 'https://www.cl...","{'scale': 'country', 'identifier': {'id': 'AFG...",2005-01-01,"{'gas': 'kyotogases', 'value': 15.1093, 'unit'..."
1,"{'name': 'wri-unfccc', 'link': 'https://www.cl...","{'scale': 'country', 'identifier': {'id': 'AFG...",2005-01-01,"{'gas': 'CH4', 'value': 9.29649, 'unit': {'uni..."
2,"{'name': 'wri-unfccc', 'link': 'https://www.cl...","{'scale': 'country', 'identifier': {'id': 'AFG...",2005-01-01,"{'gas': 'N2O', 'value': 5.81281, 'unit': {'uni..."
3,"{'name': 'wri-unfccc', 'link': 'https://www.cl...","{'scale': 'country', 'identifier': {'id': 'AFG...",2005-01-01,"{'gas': 'kyotogases', 'value': 3.77615946, 'un..."
4,"{'name': 'wri-unfccc', 'link': 'https://www.cl...","{'scale': 'country', 'identifier': {'id': 'AFG...",2005-01-01,"{'gas': 'CH4', 'value': 0.7362759600000001, 'u..."
...,...,...,...,...
59939,"{'name': 'wri-unfccc', 'link': 'https://www.cl...","{'scale': 'country', 'identifier': {'id': 'ZWE...",2006-01-01,"{'gas': 'CH4', 'value': 5.363799, 'unit': {'un..."
59940,"{'name': 'wri-unfccc', 'link': 'https://www.cl...","{'scale': 'country', 'identifier': {'id': 'ZWE...",2006-01-01,"{'gas': 'CO2', 'value': -72.004849, 'unit': {'..."
59941,"{'name': 'wri-unfccc', 'link': 'https://www.cl...","{'scale': 'country', 'identifier': {'id': 'ZWE...",2006-01-01,"{'gas': 'N2O', 'value': 4.82577, 'unit': {'uni..."
59942,"{'name': 'wri-unfccc', 'link': 'https://www.cl...","{'scale': 'country', 'identifier': {'id': 'ZWE...",2006-01-01,"{'gas': 'kyotogases', 'value': 0.63168, 'unit'..."
